In [1]:
import re
import pandas as pd
import gensim
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji

D:\deniz\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('aftercorona.csv')
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site
0,2.730776e+07,50931,AppelquistP,Damn I thought I was finally in the #Matrix or...,0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
1,2.880588e+09,27578,kaarnama13,â€œNosediveâ€ episode from #blackmirror gets ...,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
2,2.370116e+09,115767,Niggvtivity,I've just watched episode S03 | E05 of Black M...,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
3,4.817271e+07,4971,DystopicRedhead,The perfect description of the #Tories and the...,4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
4,1.402080e+18,13,smash_media_us,ã€ #ONEUS Ã— #smash. ã€‘\n\nONEUS is on now s...,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json


In [3]:
df.columns

Index(['user_id', 'user_statuses', 'name', 'tweet', 'retweets', 'location',
       'created', 'followers', 'is_user_verified', 'favorite_count',
       'in_reply_to_status_id', 'keyword', 'language', 'site'],
      dtype='object')

In [4]:
def clean_complete(tweet):
    """
    tweet: pandas series
    prepares tweets complete cleaning for further lemmatization and dering embeddings
    """
    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #remove repeated charachters
    
    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    tweet = tweet.str.replace(r':-\)', ' smile')
    tweet = tweet.str.replace(r':-\(', ' sad')
    tweet = tweet.str.replace(r':-\/', ' confused')
    tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')
    tweet = tweet.str.replace('"', '')
    tweet = tweet.str.replace('ℐℓ٥ﻻﻉ√٥υ', '')
    
    #to lower case
    tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #replacing emojies with ''
    #tweet = [demoji.replace(c, ' ') for c in tweet]

    #delete punctuation
    tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [5]:
df['tweet'] = clean_complete(df.tweet)

In [6]:
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,site
0,2.730776e+07,50931,AppelquistP,damn thought finally matrix blackmirror aliens...,0,OZ,18/06/2021,130,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
1,2.880588e+09,27578,kaarnama13,â€œnosediveâ€ episode blackmirror gets closer...,0,Aage se right,18/06/2021,669,False,6,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
2,2.370116e+09,115767,Niggvtivity,watched episode black mirror blackmirror tvtime,0,"Bonneuil-sur-Marne, France",18/06/2021,795,False,0,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
3,4.817271e+07,4971,DystopicRedhead,perfect description tories voters favourite sc...,4,United Kingdom,18/06/2021,2117,False,8,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json
4,1.402080e+18,13,smash_media_us,ã€ oneus smash ã€‘oneus smash â£ï¸new song ...,0,NaN,18/06/2021,762,False,5,NaN,#blackmirror,en,date post corona\2021-06-18_en_#blackmirror.json


In [7]:
df.to_csv('to_hive_aftercorona.csv', sep=',', header=False)